# LSSTCam pointing errors

Craig Lage - 16-Apr-25

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from astropy.coordinates import EarthLocation
location = EarthLocation.of_site('Rubin:Simonyi')


In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation', 
                                             'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'

In [ ]:
startDay = 20250527
endDay = 20250601

ras = []
decs = []
trueRas = []
trueDecs = []
side_times = []
dayObs = startDay
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['science', 'acq']:
            continue
        try:
            calExp = butler.get('preliminary_visit_image', detector=94, visit=record.id, instrument=instrument)
            rawExp = butler.get('raw', detector=94, exposure=record.id, instrument=instrument)
            md = rawExp.getMetadata()
            cWcs = calExp.getWcs()
            rWcs = rawExp.getWcs()
            rawSkyCenter = rWcs.getSkyOrigin()
            calExpSkyCenter = cWcs.pixelToSky(rWcs.getPixelOrigin())
            trueRa = calExpSkyCenter.getRa().asDegrees()
            trueDec = calExpSkyCenter.getDec().asDegrees()
            trueRas.append(trueRa)
            trueDecs.append(trueDec)
            ra = md['RASTART']
            dec = md['DECSTART']
            ras.append(ra)
            decs.append(dec)
            time = Time((md['MJD-BEG'] + md['MJD-END']) / 2.0,  scale='tai', format='mjd')
            side_time = time.sidereal_time('mean', longitude=location)
            side_times.append(side_time)
            print(f"{record.id}, ra = {ra:.6f}, trueRa = {trueRa:.6f}, dec = {dec:.6f}, trueDec = {trueDec:.6f}")
        except:
            print(f"{record.id} failed!")
            continue
    print(dayObs, len(ras))
    dayObs = calcNextDay(dayObs)




In [ ]:
filename = "/home/c/cslage/u/LSSTCam/data/pointing_results_ra_dec_27may25-01jun25.pkl"
with open(filename, 'wb') as f:
    pkl.dump([ras, decs, trueRas, trueDecs, side_times], f)


In [ ]:
filename = "/home/c/cslage/u/LSSTCam/data/pointing_results_ra_dec_27may25-01jun25.pkl"
with open(filename, 'rb') as f:
    [ras, decs, trueRas, trueDecs, side_times] = pkl.load(f)
len(els)

In [ ]:
print(len(ras), len(decs), len(trueRas), len(trueDecs), len(side_times))

# Generate a tpoint input file with these RaDec values

In [ ]:
outfilename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_RaDec_01Jun25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,June 1, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write("-30 14 40.2\n")
for i in range(len(els)):
    h = side_times[i].hms.h
    m = side_times[i].hms.m + side_times[i].hms.s / 60.0
    outfile.write(f"{trueRas[i]:.9f}\t{trueDecs[i]:.9f}\t{ras[i]:.9f}\t{decs[i]:.9f}\t{h:02d}\t{m:.4f}\n")
outfile.write("END\n")
outfile.close()

